In [ ]:
import pandas as numpy
import numpy as np
import matplotlib.pyplot as plt
from skimage.color import rgb2lab,lab2rgb
from skimage.io import imread,imshow
import shutil
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input, LeakyReLU
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential
import math

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
import zipfile
with zipfile.ZipFile('/content/drive/My Drive/ML/Brain_tumour/Brain_Tumour_dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/drive/My Drive/ML/Brain_tumour')

In [ ]:
ROOT_DIR="/content/drive/MyDrive/ML/Brain_tumour/brain_tumor_dataset"
number_of_images={}
for dir in os.listdir(ROOT_DIR):
  number_of_images[dir]=len(os.listdir(os.path.join(ROOT_DIR,dir)))
number_of_images.items()

we will create a training folder


In [ ]:
src="/content/drive/MyDrive/ML/Brain_tumour/brain_tumor_dataset"
if not os.path.exists("./train"):
  os.mkdir("./train")

  for dir in os.listdir(src):
    os.makedirs("./train/"+dir)


    for img in np.random.choice(a=os.listdir(os.path.join(src,dir)),
                                  size=(math.floor(70/100*number_of_images[dir])),
                                  replace =False ):
      O=os.path.join(src,dir,img)
      D=os.path.join("./train",dir)
      shutil.copy(O,D)
      os.remove(O)

else:
  print("folder exist")

In [ ]:
src="/content/drive/MyDrive/ML/Brain_tumour/brain_tumor_dataset"
if not os.path.exists("./valid"):
  os.mkdir("./valid")

  for dir in os.listdir(src):
    os.makedirs("./valid/"+dir)


    for img in np.random.choice(a=os.listdir(os.path.join(src,dir)),
                                  size=(math.floor(15/100*number_of_images[dir])),
                                  replace =False ):
      O=os.path.join(src,dir,img)
      D=os.path.join("./valid",dir)
      shutil.copy(O,D)
      os.remove(O)

else:
  print("folder exist")

In [ ]:
src="/content/drive/MyDrive/ML/Brain_tumour/brain_tumor_dataset"
if not os.path.exists("./test"):
  os.mkdir("./test")

  for dir in os.listdir(src):
    os.makedirs("./test/"+dir)


    for img in np.random.choice(a=os.listdir(os.path.join(src,dir)),
                                  size=(math.floor(15/100*number_of_images[dir])),
                                  replace =False ):
      O=os.path.join(src,dir,img)
      D=os.path.join("./test",dir)
      shutil.copy(O,D)
      os.remove(O)

else:
  print("folder exist")

In [ ]:
number_of_images={}
for dir in os.listdir(ROOT_DIR):
  number_of_images[dir]=len(os.listdir(os.path.join(ROOT_DIR,dir)))
number_of_images.items()

Model Build


In [ ]:
from keras.layers import Conv2D,MaxPool2D, Dropout , Flatten ,Dense ,BatchNormalization, GlobalAvgPool2D
from keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import MobileNet, preprocess_input
import keras

In [ ]:
model =Sequential()

model.add(Conv2D(filters=16, kernel_size=(3,3), activation="relu", input_shape =(224,224,3)))

model.add(Conv2D(filters=36, kernel_size=(3,3), activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(units=64 ,activation ='relu'))

model.add(Dropout(rate=0.25))
model.add(Dense(units=1 ,activation ='sigmoid'))

model.summary()

In [ ]:
model.compile(optimizer='adam', loss=keras.losses.binary_crossentropy, metrics=['accuracy'])

In [ ]:
def preprocessingImage1(path):
  """
  input : Path
  output : Pre processed images
  """

  image_data = ImageDataGenerator(zoom_range= 0.2, shear_range= 0.2, rescale=1/255, horizontal_flip=True)
  image=image_data.flow_from_directory(directory= path, target_size= (224,224), batch_size= 32, class_mode = "binary")

  return image

In [ ]:
path ='/content/train'
train_data = preprocessingImage1(path)

In [ ]:
def preprocessingImage2(path):
  """
  input : Path
  output : Pre processed images
  """

  image_data = ImageDataGenerator(rescale=1/255)
  image=image_data.flow_from_directory(directory= path, target_size= (224,224), batch_size= 32, class_mode = "binary")

  return image

In [ ]:
path ='/content/valid'
val_data = preprocessingImage2(path)

In [ ]:
path ='/content/test'
test_data = preprocessingImage2(path)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

es=EarlyStopping(monitor='val_accuracy',min_delta=0.01,patience=3,verbose=1,mode ='auto')
mc= ModelCheckpoint(monitor="val_accuracy",filepath="./bestmodel.keras", verbose=1 , save_best_only=True , mode ='auto')
cd= [es,mc]

In [ ]:

hs = model.fit(train_data,
               steps_per_epoch=8,
               epochs=50,
               verbose=1,
               validation_data=val_data,
               validation_steps=16,
               callbacks=cd)



In [ ]:
# Model graphical

In [ ]:
h=hs.history
h.keys()

In [ ]:
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'],c ='red')
plt.show()

In [ ]:
plt.plot(h['loss'])
plt.plot(h['val_loss'],c ='red')
plt.show()

In [ ]:
# Accuracy

In [ ]:
from keras.models import load_model

In [ ]:
model =load_model("/content/bestmodel.keras")

In [ ]:
cc = model.evaluate(test_data)[1]  # Simply use model.evaluate()

print(f"accuracy {cc*100:.2f}%")

In [ ]:
from keras.preprocessing.image import load_img,img_to_array

In [ ]:
path = "/content/test/yes/Y21.jpg"
img = load_img(path, target_size=(224, 224))  # Load image and resize
input_arr = img_to_array(img) / 255.0         # Convert to array and normalize

# Display the image
plt.imshow(input_arr)
plt.axis('off')  # Optional: Hide axis for clean display
plt.show()

# Add batch dimension
input_arr = np.expand_dims(input_arr, axis=0)

# Get the predicted probability
pred = model.predict(input_arr)[0][0]

# Convert probability to binary class (0 or 1) using a threshold
pred_class = 1 if pred > 0.5 else 0

print(f"Predicted probability: {pred}")
if(pred_class==0):
  print("No Tumor")
else:
  print("Tumor")



In [ ]:
train_data.class_indices

BY mobilenet model


In [ ]:
 import numpy as np
 import matplotlib.pyplot as plt
 from keras.layers import Flatten, Dense
 from keras.models import Model, load_model
 from keras.applications.mobilenet import MobileNet, preprocess_input
 import keras



In [ ]:
base_model=MobileNet(input_shape=(224,224,3), include_top=False)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
X=Flatten()(base_model.output)
X=Dense(units=1, activation='sigmoid')(X)

model=Model(base_model.input,X)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss=keras.losses.binary_crossentropy, metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

es=EarlyStopping(monitor='val_accuracy',min_delta=0.01,patience=10,verbose=1,mode ='auto')
mc= ModelCheckpoint(monitor="val_accuracy",filepath="./bestmodel.keras", verbose=1 , save_best_only=True , mode ='auto')
cd= [es,mc]

In [ ]:
hs = model.fit(
    train_data,                 # Pass the training generator or dataset here
    steps_per_epoch=10,         # Adjust steps_per_epoch based on your data size
    epochs=30,                  # Set the number of epochs
    verbose=1,                  # Verbosity mode (1 = progress bar, 0 = silent)
    validation_data=val_data,   # Validation dataset/generator
    validation_steps=16,        # Adjust based on validation set size
    callbacks=cd                # Add any callbacks, e.g., EarlyStopping, ModelCheckpoint
)


In [ ]:
model=load_model("/content/bestmodel.keras")

In [ ]:
acc=model.evaluate(test_data)[1]
print(f"Accuracy {acc*100:.2f}%")

In [ ]:
h=hs.history
h.keys()

In [ ]:
from tensorflow.keras.preprocessing import image  # Import the image module
import numpy as np

In [ ]:
path="/content/valid/yes/Y103.jpg"
img=image.load_img(path,target_size=(224,224))
i=image.img_to_array(img)/255
input_arr=np.array([i])
input_arr.shape

pred=np.argmax(model.predict(input_arr))
if pred==0:
  print("No Tumor")
else:
  print("Tumor")

plt.imshow(input_arr[0])
plt.title("input image")
plt.show()

